# GEOL 593: Seismology and Earth Structure

## Lab assignment 7: Seismic tomography

Today's lab focuses on seismic tomography, which is one of the primary ways in which we image the structural variations in the Earth's subsurface. In particular, we will focus on using **travel-time tomography** to image seismic wave speed variation, under the assumption of **ray-theory**. The ray-theoretical approximation assumes that seismic energy travels along infinitely narrow paths (called rays) connecting seismic sources and receivers. This is technically an *infinite-frequency* approximation. In other words, ray theory would be correct if seismic waves had infinite frequency. While this is not true (and in many cases finite-frequency effects should not be ignored) ray-theory is still a useful approximation. In fact, until reletively recently, the majority of global tomography models were based on ray theory, and it still finds use in many applications today.

According to ray-theory, the travel time $T$ of a seismic wave is modeled by a line integral along the ray

$
\displaystyle
T = \int_{ray} u(s)ds
$

where $u(s)$ is the 'slowness' (inverse of seismic wave speed), and $ds$ is an increment along the ray path

One important choice we need to make when doing tomography is how to parameterize the model. The simplest choice (and what we use here) is to split the model into cells or blocks. The problem then is to solve for the value of $u$ in each block. Other choices might include a set of continuous basis functions (e.g., spherical harmonics), in which case you would solve for the coefficients of each basis function. 

In travel time tomography, we typically have many hundreds or thousands of travel time observations, and collect all of the observations into a vector. When using a block parameterization, the $i^{th}$ travel time observation can be written

$
\displaystyle
T_i = \sum_{j=1}G_{ij}u_j
$

where $G_{ij}$ is the distance that the $i^{th}$ ray travels in the $j^{th}$ block

If we call the travel time vector $\mathbf{d}$, and the slowness vector $\mathbf{m}$, we can write the above equation in the usual form

$G \mathbf{m} = \mathbf{d}$

which we have experience solving.

###  <font color='red'>Question 1 </font> 

Above, we described how to set up the tomography problem as a linear inverse problem of the form $G \mathbf{m} = \mathbf{d}$. In reality, tomography is not an inherently linear problem, and we made some assumptions to *linearize* the problem. 

**Discuss some of these assumptions. When would they break down?**

Hint: For non-linear problems, the G matrix depends on the model vector. Here,  the G matrix encodes the rays that are used to calculate the path lengths in each cell. What would cause you to need to recalculate the rays? See also Chapter 20 of the Ammon textbook (i.e., section 20.1.2). 


#### Answer question 1 in this Markdown cell.


### Toy tomography problem

Below, we are going to explore tomographic imaging using a 'toy problem', in which we calculate travel times for a known model (referred to as our target model), and invert the the travel times using tomography in order to see how accurately we can image the model. This will give us insight into the ideal conditions for imaging, and how the process of tomography can distort the true image. Our example will consist of recovering a 2D image using straight-ray tomography (i.e., the paths between sources and receivers are assumed to be straight lines). In terms of programming, the key part of this problem consists of calculating the distance that each ray travels in each block of the model (i.e., calculating the G matrix), and calculating the travel times for our target model (i.e., calculating the data vector). The most challenging parts have been coded below.

The function `setup_tomo` is provided to set up the tomographic inverse problem. It takes several inputs and outputs, so spend some time studying the code. The key inputs you must provide are `size_x`, `size_y`, and `n_paths`. `size_x` and `size_y` are the number of blocks to divide the model into in the x-direction, and y-direction, respectively. Therefore, the total number of model parameters that we will be solving for is `size_x * size_y`. `n_paths` is the total number of ray paths that will be used for imaging. The source and receiver locations will be randomized each time you call `setup_tomo`. It is unneccessary to provide additional input parameters because they are initialized with default values. By default, our imaging domain is a square with dimensions 100 km x 100 km. The target model is a slowness map representing the University of Illinois logo. The file is located on the github repository under `data/lab_07/target_model.npy`.

In [3]:
#imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import lsmr
from scipy.interpolate import interp2d
from scipy.interpolate import RegularGridInterpolator

import warnings
warnings.filterwarnings('ignore') #dangerous to do this, but scipy was being annoying.

In [13]:
def setup_tomo(size_x,size_y,n_paths,x_min=0.,x_max=100.,y_min=0.,y_max=100.,u_ref=1./3.0):
    '''
    sets up a simple 'toy' tomography problem
    
    inputs
        size_x: number of model parameters in x dimension
        size_y: number of model parameters in y dimension
        n_paths: the number of ray paths
        x_min: minimum x-coordinate (default = 0)
        x_max: maximum x-coordinate (default = 100)
        y_min: minimum y-coordinate (default = 0)
        y_max: maximum y-coordinate (default = 100)
        u_ref: slowness of reference model
    returns
        G: the G-matrix (system matrix)
        data_obs: vector containing the "observed" travel time data (i.e., the d in Gm=d)
        data_obs_ref: vector containing the travel time data for the 'reference model' (ie., a model with a constant slowness equal to mod_ref_val)
        mod_target: the model that you are attempting to recover
        srcs_and_recs: list of sources and receiver locations
    '''

    #step 1. define reference model
    mod_ref = np.zeros((size_x,size_y))
    mod_ref += u_ref #target model has a slowness of 1./3. s/km (3 km/s in velocity)

    #step 2. create target model (load model and interpolate to same parameterization)
    target_high_res = np.load('../data/lab_07/target_model.npy')
    x_ = np.linspace(x_min,x_max,target_high_res.shape[0],)
    y_ = np.linspace(x_min,x_max,target_high_res.shape[1],)
    x_i = np.linspace(0,100,size_x)
    y_i = np.linspace(0,100,size_y)
    f = interp2d(y_, x_, target_high_res, kind='cubic') #may cause deprecation warning... ok to ignore
    mod_target = f(x_i,y_i)

    #step 3. define the sources and receivers
    srcs_x = np.random.random(n_paths)*x_max
    srcs_y = np.random.random(n_paths)*y_max
    recs_x = np.random.random(n_paths)*x_max
    recs_y = np.random.random(n_paths)*y_max
    srcs_and_recs = [srcs_x,srcs_y,recs_x,recs_y]

    #step 4 calculate G (needs to be recalculated whenever you change sources or receivers)
    G = calc_G(srcs_x = srcs_x, srcs_y = srcs_y, recs_x = recs_x, recs_y = recs_y,size_x=size_x,size_y=size_y)

    #step 5. calculate synthetic data
    data_obs_ref = np.dot(G,mod_target.flatten())
    data_obs = np.dot(G,mod_target.flatten())
    
    return G, data_obs, data_obs_ref, mod_target, srcs_and_recs

#tomography helper functions
def find_ind(x,y,x0,x1,y0,y1,npts_x,npts_y):
    #------------------------------------------------------------
    # find model index of location along ray
    #
    #inputs
    #    x: x cooridinate of point
    #    y: y coordinate of point
    #    x0: x-coordinate of left margin of model
    #    y0: y-coordinate of bottom margin of model
    #    x1: x-coordinate of right margin of model
    #    y1: y-coordinate of top margin of model
    #    npts_x: number of model points in x
    #    npts_y: number of model points in y
    #outputs:
    #    ind_x, ind_y: tuple with x and y index of point
    #------------------------------------------------------------
    delta_x = (x1 - x0) / npts_x
    delta_y = (y1 - y0) / npts_y
    ind_x = int((x - x0)/delta_x)
    ind_y = int((y - y0)/delta_y)
    return(ind_x,ind_y)
    
def calc_G(srcs_x,srcs_y,recs_x,recs_y,size_x, size_y):
    #------------------------------------------------------------
    # calculate G matrix
    #
    #inputs
    #    srcs_x: list or numpy array of x coordinates of sources
    #    srcs_y: list or numpy array of y coordinates of sources
    #    recs_x: list or numpy array of x coordinates of receivers
    #    recs_y: list or numpy array of y coordinates of receivers
    #    size_x: number of model parameters (blocks) in x direction
    #    size_y: number of model parameters (blocks) in y direction
    #outputs:
    #    G: the G matrix (size M,N, where M is total number of parameters, and N is number of paths)
    #------------------------------------------------------------
    
    G = []
    path_dists = []
    
    for i in range(0,len(srcs_x)):
        
        row = np.zeros((size_x,size_y))
        npts_path = 1000
        path_x = np.linspace(srcs_x[i],recs_x[i],npts_path)
        path_y = np.linspace(srcs_y[i],recs_y[i],npts_path)
        path_dist = 0
        
        for j in range(0,npts_path-1):
        
            #find index of currect point along path
            ind_x,ind_y = find_ind(path_x[j],path_y[j],x0=0,x1=100,y0=0,y1=100,npts_x=size_x,npts_y=size_y)
        
            #find distance between last point
            dist = np.sqrt((path_x[j+1] - path_x[j])**2 + (path_y[j+1] - path_y[j])**2)
            row[ind_x,ind_y] += dist
            path_dist += dist
            
        G.append(row.flatten())
        path_dists.append(path_dist)
    
    G = np.array(G)   
    path_dists = np.array(path_dists)
    return G

def get_adjacent_indices(i, j, m, n):

    adjacent_indices = []
    if i > 0:
        adjacent_indices.append((i-1,j))
    if i+1 < m:
        adjacent_indices.append((i+1,j))
    if j > 0:
        adjacent_indices.append((i,j-1))
    if j+1 < n:
        adjacent_indices.append((i,j+1))
    return adjacent_indices

def gen_smoothness_matrix(size_x,size_y):
    '''
    for the inverse problem... generates a matrix used for smoothing. 
    pretty crude implementation
    '''
    
    smoothness_matrix = np.zeros((size_x**2,size_y**2))
    count = 0
    for i in range(0,size_x):
        for j in range(0,size_y):
            adj_inds = get_adjacent_indices(i,j,m=size_x,n=size_y)

            row = np.zeros((size_x,size_y))
            row[i,j] = -2.0
            for k in range(0,len(adj_inds)):
                row[adj_inds[k]] = 0.5
                smoothness_matrix[:,count] = row.flatten()
            count += 1
            
    return smoothness_matrix

In [5]:
def plot_ray_density(srcs_and_recs,size_x, size_y,plot_paths=True):
    #------------------------------------------------------------
    # plots a map of ray coverage
    #
    #inputs
    #    srcs_and_recs: list containing source and receiver arrays (returned by setup_tomo)
    #    size_x: number of model parameters (blocks) in x direction
    #    size_y: number of model parameters (blocks) in y direction
    #    plot_paths: plot paths (True/Fase)
    #------------------------------------------------------------
    ray_map = np.zeros((size_x,size_y))
    srcs_x = srcs_and_recs[0]
    srcs_y = srcs_and_recs[1]
    recs_x = srcs_and_recs[2]
    recs_y = srcs_and_recs[3]
    
    for i in range(0,len(srcs_x)):
        path_map = np.zeros((size_x,size_y))
        
        npts_path = 1000
        path_x = np.linspace(srcs_x[i],recs_x[i],npts_path)
        path_y = np.linspace(srcs_y[i],recs_y[i],npts_path)
        path_dist = 0
        
        for j in range(0,npts_path-1):
        
            #find index of currect point along path
            ind_x,ind_y = find_ind(path_x[j],path_y[j],x0=0,x1=100,y0=0,y1=100,npts_x=size_x,npts_y=size_y)
            if path_map[ind_x,ind_y] == 0.0:
                path_map[ind_x,ind_y] += 1
                
        ray_map += path_map
        
        if plot_paths:
            path_x_ = np.linspace(srcs_x[i],recs_x[i],2)
            path_y_ = np.linspace(srcs_y[i],recs_y[i],2)
            plt.plot(path_x_,path_y_,c='k',linewidth=0.5,alpha=0.5,zorder=99)
            #plt.plot(path_y,path_x,c='k')
        
    plt.imshow(np.flipud(ray_map.T),cmap='jet',extent=[0,100,0,100])
    plt.colorbar(label='# of paths')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

###  <font color='red'>Question 2 </font>

i) Run `setup_tomo` to set up the tomography problem with 31 blocks in both the x and y dimensions (`size_x` and `size_y` both equal 31), and using `n_paths = 200`.

ii) Plot the target model `mod_target` (i.e., the model that you are attempting to tomographically image) using the `plt.imshow()` command (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). You can change to colormap used to plot the data by setting the `cmap` argument. The `RdYlBu` colormap is probably most appropriate for UIUC fans. Be sure to include a colorbar with `plt.colorbar()`. Note, the values are in *slowness* (s/km). If you plot `1/mod_target` it would be in velocity (km/s).

In [6]:
#Answer Question 2 here.

###  <font color='red'>Question 3 </font>

Use the function `plot_ray_density` (provided above) to create a plot of the ray path coverage of your problem. The function takes a list of the source and receiver locations (`srcs_and_recs`), which is one of the items returned by `setup_tomo`. It also needs `size_x` and `size_y` (i.e., the number of model parameters in x and y).

**Based on the ray-density plot, where do you expect your model to have the best and worst resolution?** 

In [7]:
#Answer Question 3 here.

### Least squares inversion

As you have seen, the least squares solution to the problem $G \mathbf{m} = \mathbf{d}$ is 

$
\displaystyle
\mathbf{m_{est}} = \left[G^{T}G\right]^{-1} G^{T}\mathbf{d}
$

In previous labs you have used the function `np.linalg.lstsq` to solve the linear least squares problem, but here we are going to code the solution directly.

An example of how this looks is given below, where the G-matrix is named G and the data vector is named data_obs. Remember that we use `np.dot` to perform matrix-vector multiplication.

`gtg = np.dot(G.T,G)`

`gtg_inv = np.linalg.inv(gtg)`

`gtg_inv_gt = np.dot(gtg_inv,G.T)`

`m_est = np.dot(gtg_inv_gt,data_obs)`

**Important note: m_est (the recovered model) is a vector of model parameters. In order to plot it, you need to 'reshape' it to a 2D numpy array with the correct number of model parameters in x and y. For example, 
`m_est = m_est.reshape(size_x, size_y)`**

###  <font color='red'>Question 4 </font> 

i) Code up the least squares solution to the tomography problem (as is shown above). When the number of paths `n_paths` is set to 200 and `size_x = size_y = 31` you should get the following error:

`LinAlgError: Singular matrix`

**Why are you are getting this error? In other words, what is a singular matrix, and why is your matrix singular?**

ii) To 'fix' this error, there are a couple of things that we can try to do. First, we could decrease the number of model parameters (ie. make the blocks bigger), so that no blocks go unsampled by rays. Second, we can increase the number of rays until all blocks are sampled with the current model parameterization. We will try the latter. 

**Increase the number of ray paths by changing the `n_paths` parameter in `setup_tomo`, and try to find the least squares solution. How many paths do need to include before you no longer get the `Singular matrix` error? Don't increase the number of rays too high (e.g., you shouldn't need several thousand) Note, the source and receiver locations are randomized, so there is no exact correct answer.**

iii) **Once you have a solution for *m_est*, plot it!** *Note, keep an eye on the range of values present in the model by plotting a colorbar. By default the colormap is auto-scaled based on the minimum and maximum values present in the array that is being plotted. You can override this by setting the `vmin` and `vmax` arguments in `plt.imshow`*

In [8]:
#Answer Question 4 here.

### Regularization

#### Norm damping

In many cases, we can not acheive ideal ray path coverage, and we are left with a situation in which the matrix $G^T G$ does not have an inverse. In this case, there are *infintely many solutions*, and to pick a best solution, we must impose some conditions, or apply 'regularization'. One of the simplest and most common types of regularization is to find the "smallest" model capable of fitting the data, as measured by the L-2 norm of the model vector. The L-2 norm is measured as 

$
\displaystyle
\lVert \mathbf{m} \rVert_2 = \sqrt{\sum_{i=1}^N |m_i|^2}
$

The problem is then to minimize the quantity

$
\displaystyle
\lVert G\mathbf{m} - \mathbf{d} \rVert_2 + \lambda^2\lVert \mathbf{m} \rVert_2
$

where the first term is the misfit between the predicted data and observations, and the second term relates to the model size. The parameter $\lambda$ is called the *damping factor*, and needs to be chosen. Larger values of $\lambda$ imply heavier damping. When an inversion minimizes both the data misfit and norm of the model (L-2 or otherwise), the type of regularization is referred to as 'norm-damping'. Also, you may see this referred to as a type of *Tikhanov regularization*. The solution to the norm damping problem has the following form

$
\displaystyle
\mathbf{m_{est}} = \left[G^{T}G + \lambda^2 I \right]^{-1} G^{T} \mathbf{d}
$

where $I$ is the *identity matrix* (i.e., a square matrix with the dimensions of $G^TG$, with 1's down the diagonal, and 0's everywhere else).


Instead of minimizing the total L-2 norm of a model, we may wish to minimize the *difference* between a model and some reference model $\mathbf{m_{ref}}$. Similar to above, this case has the solution

$
\displaystyle
\mathbf{m_{est}} = \mathbf{m_{ref}} + \left[G^{T}G + \lambda^2 I \right]^{-1} G^{T} \left[ \mathbf{d} - G \mathbf{m_{ref}} \right]
$


#### Smoothing

The other quantity that we may want to minimize is the 'roughness' of the model. In other words, we may prefer models that smoothly vary in space. Another way of thinking about this is that we want to minimize the second spatial derivative of the model, sometimes called the *Laplacian operator*, $\nabla^2$. If we define $L$ as a finite-difference approximation of the Laplacian, the quantity we aim to minimize is

$
\displaystyle
\lVert G\mathbf{m} - \mathbf{d} \rVert_2 + \epsilon^2\lVert L \mathbf{m} \rVert_2
$

where $\epsilon$ is the smoothness damping parameter (higher $\epsilon$ = more smoothness imposed). This has a similar solution as to what is shown above for norm damping.

#### Implementation
Below, the function `regularized_inversion` provides an implementation of the tomographic inverse problem that includes both norm damping and smoothing. The function takes several inputs. They are as follows: 

    G: G matrix (returned by setup_tomo)
    data_obs: 'observed' travel time vector (returned by setup_tomo)
    size_x: number of model parameters in x dimension
    size_y: number of model parameters in y dimension
    damping_factor: lambda
    smoothing_factor: epsilon

In [9]:
def regularized_inversion(G,data_obs,size_x,size_y,damping_factor=0,smoothing_factor=0,u_ref=1./3.0):
    '''
    inputs
        G: G matrix
        data_obs: 'observed' travel time vector
        size_x: number of model parameters in x dimension
        size_y: number of model parameters in y dimension
        damping_factor: lambda
        smoothing_factor: epsilon
        u_ref: slowness of reference model (s/km)
    outputs
        m_est: result
    '''
    m_ref = np.zeros((size_x,size_y))
    m_ref += u_ref #target model has a slowness of 1./3. s/km (3 km/s in velocity)
    d_ref = np.dot(G,m_ref.flatten()) #predict data for reference model
    dd = data_obs - d_ref #difference between 'observed' data and data predicted for reference model
    
    smoothness_matrix = gen_smoothness_matrix(size_x,size_y)

    gtg = np.dot(G.T,G)
    I = np.eye(gtg.shape[0])*damping_factor
    Wm = np.dot(smoothness_matrix.T,smoothness_matrix) * smoothing_factor
    gtg_inv = np.linalg.inv(gtg + I + Wm)
    gtg_inv_gt = np.dot(gtg_inv,G.T)
    m_est = np.dot(gtg_inv_gt,dd)
    m_est = m_est.reshape(size_x,size_y)
    m_est += m_ref
    
    return m_est

###  <font color='red'>Question 5 </font> 

Let's do some inversions! First, let's revisit the problem that caused us to get a `Singular Matrix` error. To do this, run, `setup_tomo` again below with parameters `size_x = 31, size_y=31` and `n_paths = 200`.

i) peform the inversion using the `regularized_inversion` function, and setting `damping_factor=1` and `smoothing_factor=0` (i.e., only use norm damping). Did the inversion work? Plot the resulting model.

ii) peform the inversion again, but set `damping_factor=0` and `smoothing_factor=1` (i.e., only use smoothing). Plot the resulting model.

*Note, if you run the `regularized_inversion` function with both `damping_factor=0` and `smoothing_factor=0`, you will likely still get a singular matrix error! (feel free to try this)*

In [10]:
#Answer Question 5 here.

### L-curve analysis

The choice of regularization parameters (i.e., amount of smoothing and damping) is important, but it is also subjective. How do we determine what are the best choices?

One common way to determine the regularization parameters is an "L-curve" test, in which you perform the inversion for different choices of regularization parameters, and pick the parameters that optimize the trade-off between the model size (i.e., $\lVert \mathbf{m} \rVert_2$), and the model misfit (i.e., $\lVert G \mathbf{m}  -\mathbf{d} \rVert_2$), where $\lVert \rVert_2$ is the L-2 norm. If you plot the model size against the model misfit for different regularization parameters, you will often see that it forms an L-shape. The 'best' set of regularization parameters are the ones that correspond to the model nearest to the bend in the L. 

In [11]:
def calc_misfit(G,m_est,data_obs):
    data_pred = np.dot(G,m_est.flatten())
    misfit = data_obs - data_pred
    l2_misfit = np.linalg.norm(misfit)
    return l2_misfit

###  <font color='red'>Question 6 </font> 

For the tomography problem you set up above (i.e., `size_x = 31, size_y=31` and `n_paths = 200`), perform an L-curve test to find the best damping parameter, while fixing the smoothing parameter to 1. To do this, loop through a range of damping parameters from 0 to 10, and solve the inverse problem for each case (i.e., run the `regularized_inversion` function). Each time you solve the inverse problem in the loop, you need to calculate the 'size' of the model, and the total misfit predicted for the model. The model size (L-2 norm) can be calculated with `np.linalg.norm(m_est)`. The misfit for the predicted model can be found using the helper function above `calc_misfit`, which takes the G matrix `G`, the tomographic model `m_est`, and the observed data `data_obs`. It returns the L-2 norm of the misfit vector. Every time you calculate $\lVert \mathbf{m} \rVert_2$ and $\lVert G \mathbf{m}  -\mathbf{d} \rVert_2$, append them to a list (e.g., `model_sizes` and `model_misfits`). In the code block below, the problem has been started for you. 

**i) Plot your lists of model size and misfit against eachother to create the "L-curve".** 

**ii) What is the best choice of damping parameter?** There is no exact correct answer, but justify your choice.

In [12]:
#Answer Question 6 here.
smoothing = 1.0
damping = np.arange(0,11,1)

model_sizes = []
model_misfits = []